In [4]:
import numpy as np
import skimage
import utils
import pathlib

In [5]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
    Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
    The function takes in a grayscale image and outputs a threshold value.

    Args:
        im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
    Returns:
        int: the computed thresholding value
    """
    assert im.dtype == np.uint8
    
    hist, _ = np.histogram(im, bins=256, range=(0, 256))
    cumulative_sum = np.cumsum(hist)
    cumulative_mean = np.cumsum(hist * np.arange(256))
    
    global_mean = cumulative_mean[-1] / cumulative_sum[-1]
    between_class_variance = np.zeros(256)
    
    for t in range(256):
        weight_bg = cumulative_sum[t]
        weight_fg = cumulative_sum[-1] - weight_bg
        if weight_bg == 0 or weight_fg == 0:
            continue
        mean_bg = cumulative_mean[t] / weight_bg
        mean_fg = (cumulative_mean[-1] - cumulative_mean[t]) / weight_fg
        between_class_variance[t] = weight_bg * weight_fg * (mean_bg - mean_fg) ** 2
    
    # threshold is the value that maximizes the between class variance
    threshold = int(np.argmax(between_class_variance))
    return threshold


In [7]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

        


Reading image: images\thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed\thumbprint-segmented.png
Reading image: images\rice-shaded.png
Found optimal threshold: 134
Saving image to: image_processed\rice-shaded-segmented.png
